In [61]:
import cv2
import numpy as np
import pandas as pd
from keras.models import Model
from keras.applications import EfficientNetB0
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Dropout
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, precision_recall_fscore_support, confusion_matrix
from keras.utils import to_categorical
import tensorflow as tf

In [62]:
# One Hot encode the labels

data = "../Data/data_directory.csv"

df = pd.read_csv(data)

print(len(df))

# Drop the rows where Grade is I
df = df[df["Grade"] != "I"]

df["Grade"] = df["Grade"].map({"A": 0, "B": 1, "C": 2, "D": 3})

df["Pixels"] = ""

print(df.head())

9125
                             Image  Grade Pixels
0       a_IMAGE_001_left_ankle.jpg      2       
1        a_IMAGE_001_left_calf.jpg      2       
2  a_IMAGE_001_left_high_thigh.jpg      2       
3   a_IMAGE_001_left_low_thigh.jpg      3       
4  a_IMAGE_001_left_metatarsal.jpg      3       


In [63]:
# Turn the images into pixel values

image_folder = "../Data/images"

train_ratio = 0.85
val_ratio = 0.15

for index, row in df.iterrows():
    img_path = image_folder + "/" + row["Image"]
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224, 224))
    img = img.astype(np.float32)
    df.at[index, "Pixels"] = img
df.head()

,Image,Grade,Pixels
0,a_IMAGE_001_left_ankle.jpg,2,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
1,a_IMAGE_001_left_calf.jpg,2,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
2,a_IMAGE_001_left_high_thigh.jpg,2,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
3,a_IMAGE_001_left_low_thigh.jpg,3,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
4,a_IMAGE_001_left_metatarsal.jpg,3,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."


In [64]:
# Split the data into train, validation, and test sets

# Drop the Image column
df = df.drop(columns=["Image"])

In [65]:
# Split the data
train, test = train_test_split(df, test_size=0.15, random_state=42)

train, val = train_test_split(train, test_size=0.1, random_state=42)

X_train = np.array(train["Pixels"].tolist())
X_val = np.array(val["Pixels"].tolist())
X_test = np.array(test["Pixels"].tolist())
y_train = to_categorical(train["Grade"].tolist())
y_val = to_categorical(val["Grade"].tolist())
y_test = to_categorical(test["Grade"].tolist())

In [66]:
efficient_net_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = GlobalAveragePooling2D()(efficient_net_model.output)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
predictions = Dense(4, activation='softmax')(x)
model = Model(inputs=efficient_net_model.input, outputs=predictions)

for layer in efficient_net_model.layers[-70:]:
    layer.trainable = True
# for layer in efficient_net_model.layers:
#     layer.trainable = False
    
model.compile(optimizer=Adam(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, verbose=0, min_lr=0.00001)

early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, verbose=1, restore_best_weights=True)

batch_size = 50
epochs = 100

history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val), callbacks=[early_stopping, reduce_lr])

Epoch 1/100
127/127 ━━━━━━━━━━━━━━━━━━━━ 507s 4s/step - accuracy: 0.5844 - loss: 1.2175 - val_accuracy: 0.3940 - val_loss: 1.3720 - learning_rate: 1.0000e-04
Epoch 2/100
127/127 ━━━━━━━━━━━━━━━━━━━━ 452s 4s/step - accuracy: 0.8215 - loss: 0.5318 - val_accuracy: 0.6942 - val_loss: 0.8405 - learning_rate: 1.0000e-04
Epoch 3/100
127/127 ━━━━━━━━━━━━━━━━━━━━ 452s 4s/step - accuracy: 0.8592 - loss: 0.3894 - val_accuracy: 0.7383 - val_loss: 0.7612 - learning_rate: 1.0000e-04
Epoch 4/100
127/127 ━━━━━━━━━━━━━━━━━━━━ 452s 4s/step - accuracy: 0.8893 - loss: 0.3012 - val_accuracy: 0.7568 - val_loss: 0.7534 - learning_rate: 1.0000e-04
Epoch 5/100
127/127 ━━━━━━━━━━━━━━━━━━━━ 451s 4s/step - accuracy: 0.9177 - loss: 0.2351 - val_accuracy: 0.8051 - val_loss: 0.5969 - learning_rate: 1.0000e-04
Epoch 6/100
127/127 ━━━━━━━━━━━━━━━━━━━━ 451s 4s/step - accuracy: 0.9338 - loss: 0.1781 - val_accuracy: 0.7980 - val_loss: 0.6150 - learning_rate: 1.0000e-04
Epoch 7/100
127/127 ━━━━━━━━━━━━━━━━━━━━ 451s 4s/ste

In [67]:
model_scores = model.evaluate(X_test, y_test)
print("Test Loss:", model_scores[0])
print("Test Accuracy:", model_scores[1])

39/39 ━━━━━━━━━━━━━━━━━━━━ 17s 423ms/step - accuracy: 0.8658 - loss: 0.5969
Test Loss: 0.5694004893302917
Test Accuracy: 0.8630136847496033


In [68]:
y_pred = model.predict(X_test)

print(y_pred)

39/39 ━━━━━━━━━━━━━━━━━━━━ 23s 478ms/step
[[9.98653769e-01 2.42832248e-04 7.47149970e-05 1.02864951e-03]
 [1.36746335e-11 9.99993801e-01 1.17835519e-07 6.02542741e-06]
 [9.99913454e-01 6.40981016e-05 4.76905052e-06 1.76857084e-05]
 ...
 [9.99973774e-01 4.04682403e-08 6.00949625e-06 2.02651754e-05]
 [4.55996778e-05 9.99913454e-01 1.18674334e-05 2.90784137e-05]
 [8.81353140e-01 1.12751245e-01 7.96195527e-04 5.09946560e-03]]


In [69]:
y_pred_temp = [np.argmax(y) for y in y_pred]

y_test_temp = test["Grade"].tolist()


print(classification_report(y_test_temp, y_pred_temp))

              precision    recall  f1-score   support

           0       0.91      0.92      0.92       729
           1       0.74      0.77      0.76       261
           2       0.85      0.76      0.81       199
           3       0.85      0.85      0.85        52

    accuracy                           0.86      1241
   macro avg       0.84      0.83      0.83      1241
weighted avg       0.86      0.86      0.86      1241



In [70]:

print(confusion_matrix(y_test_temp, y_pred_temp))

[[673  48   8   0]
 [ 47 202  12   0]
 [ 17  22 152   8]
 [  2   0   6  44]]


In [76]:
def pixelate_image(image):
    """
    This function takes in an image and returns the pixelated version of the image
    :param image: The path to the image
    :return: An array of the pixelated image
    """

    img = cv2.imread(image)
    img = cv2.resize(img, (224, 224))
    img = img.astype(np.float32)
    return img


In [75]:
tf.saved_model.save(model, "waveformMedModelMulti")

INFO:tensorflow:Assets written to: waveformMedModelMulti\assets


INFO:tensorflow:Assets written to: waveformMedModelMulti\assets


In [77]:
image_path = "../Data/images/a_IMAGE_001_left_ankle.jpg"

reconstructed_model_multi = tf.saved_model.load("waveformMedModelMulti")

infer_multi = reconstructed_model_multi.signatures["serving_default"]

img = pixelate_image(image_path)
img = np.expand_dims(img, axis=0)

input_data = {"inputs": img}

prediction_multi = infer_multi(**input_data)
    
prediction_multi = prediction_multi["output_0"]

print(prediction_multi)

tf.Tensor([[8.0209560e-08 4.9692658e-06 9.9996543e-01 2.9455925e-05]], shape=(1, 4), dtype=float32)


In [78]:
print(np.argmax(prediction_multi))

2
